# Crop Delivery Data Processing

This notebook aims to process crop delivery data from multiple Excel files, transform the data, and save the final combined dataset. The steps involved in this process include:

1. **Loading Excel Files**: Reading data from multiple Excel files using the `load_excel` function.
2. **Preprocessing Data**: Cleaning and transforming the data using the `preprocess_data` function.
3. **Unpivoting Data**: Reshaping the data from wide format to long format using the `unpivot_data` function.
4. **Adding Crop Column**: Adding a column to indicate the type of crop using the `add_crop_column` function.
5. **Combining Data**: Merging data from all files into a single DataFrame using the `process_files` function.
6. **Saving Data**: Exporting the final combined DataFrame to Excel and Parquet formats.

The variables used in this notebook include:
- `base_file_path`: The base directory path where the Excel files are located.
- `file_names`, `sheet_names`, `crop_names`, `headers`, `numrows`: Lists containing details of the files, sheets, crop names, headers, and number of rows to read.
- `final_df`: The final combined DataFrame containing processed data from all specified files and sheets.

In [101]:
import os
import pandas as pd

In [ ]:
def load_excel(
    file_name: str, sheet_name: str, header: str, nrows: str
) -> pd.DataFrame:
    try:
        df = pd.read_excel(
            file_name,
            sheet_name=sheet_name,
            engine="openpyxl",
            header=header,
            nrows=nrows,
        )
    except Exception as e:
        print(f"Error loading with openpyxl: {e}. Trying with xlrd...")
        df = pd.read_excel(
            file_name,
            sheet_name=sheet_name,
            engine="xlrd",
            header=header,
            nrows=nrows,
        )
    return df


def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    df.rename(columns={"Bemarkingseisoen week": "Week"}, inplace=True)
    df = df[df["Week"] != "Early Deliveries"]
    df["Month"] = pd.to_datetime(df["Week geëindig"]).dt.month.astype("int64")
    df["Day"] = pd.to_datetime(df["Week geëindig"]).dt.day.astype("int64")
    df.drop(columns=["Week geëindig", "Week"], inplace=True)
    return df


def unpivot_data(df: pd.DataFrame) -> pd.DataFrame:
    id_vars = ["Month", "Day"]
    df = df.melt(id_vars=id_vars, var_name="Year", value_name="Crop Delivery (Tonnes)")
    df["Year"] = df["Year"].str[:4]
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
    df = df.dropna(subset=["Year"])
    df["Year"] = df["Year"].astype("int64")
    df["Date"] = pd.to_datetime(df[["Year", "Month", "Day"]])
    df.drop(columns=["Month", "Day", "Year"], inplace=True)
    return df


def add_crop_column(df: pd.DataFrame, crop_name: str) -> pd.DataFrame:
    df["Crop"] = crop_name
    df["Crop Delivery (Tonnes)"] = (
        pd.to_numeric(df["Crop Delivery (Tonnes)"], errors="coerce")
        .fillna(0)
        .astype(int)
    )
    df.reset_index(drop=True, inplace=True)
    return df


def process_files(
    file_names: list, sheet_names: list, crop_names: list, headers: list, numrows: list
) -> pd.DataFrame:
    """
    Processes multiple Excel files and combines them into a single DataFrame.

    Args:
        file_names (list): List of file paths to the Excel files.
        sheet_names (list): List of sheet names to be read from each Excel file.
        crop_names (list): List of crop names to be added as a column in the DataFrame.
        headers (list): List of row numbers to use as the column names for each sheet.
        numrows (list): List of the number of rows to read from each sheet.

    Returns:
        pd.DataFrame: A combined DataFrame containing the processed data from all specified files and sheets.
    """
    final_df = pd.DataFrame()
    for file_name, sheet_name, crop_name, header, nrows in zip(
        file_names, sheet_names, crop_names, headers, numrows
    ):
        df = load_excel(file_name, sheet_name, header, nrows)
        df = preprocess_data(df)
        df = unpivot_data(df)
        df = add_crop_column(df, crop_name)
        final_df = pd.concat([final_df, df], ignore_index=True)
    return final_df

In [ ]:
# List of file names, sheet names, and crop names
file_names = [
    "20-Nov-24-SAGIS---Sojabone-Week-prod-deliveries---lewerings-2024_2025.xlsx",
    "20-Nov-24-SAGIS---Sonneblom-Week-prod-deliveries---lewerings-2024_2025.xls",
    "20-Nov-24-SAGIS---Mielies-Week-prod-deliveries---lewerings-2024-2025.xlsx",
    "20-Nov-24-SAGIS---Mielies-Week-prod-deliveries---lewerings-2024-2025.xlsx",
]
sheet_names = [
    "Sojabone - Soybeans",
    "Sonneblom - Sunflower",
    "Summary -White maize",
    "Summary -Yellow maize",
]
crop_names = [
    "Soya",
    "Sunflower",
    "White Maize",
    "Yellow Maize",
]
headers = [
    2,
    2,
    15,
    16,
]
numrows = [
    52,
    52,
    53,
    52,
]

# Base file path
base_file_path = r"C:\Users\mario\OneDrive\Documents\Work\Clients\Agnify\2. Data\SAFEX\Crop Deliveries"
os.chdir(base_file_path)

# Process files and get the final DataFrame
final_df = process_files(file_names, sheet_names, crop_names, headers, numrows)

# Save the final DataFrame to Excel and Parquet
final_df.to_excel("SAFEX Crop Deliveries.xlsx", index=False)
final_df.to_parquet("safex_crop_deliveries.parquet")